# Trabajo en grupo 2018 - Filtros de imágenes

## Versión: SIMD

### Autores:
 - Alejandro
 - Álvaro Baños Gomez 
 - Iñaki
 - Guillermo Facundo Colunga

### Enunciado
Realizar una versión monohilo con extensiones multimedia y una versión multihilo del programa anterior para aprovechar las capacidades de paralelismo y concurrencia del sistema y así obtener una ganancia de rendimiento.

El segundo programa monohilo a desarrollar consiste en el empleo de las extensiones SIMD de la arquitectura para la mejora del rendimiento, para lo cual en esta parte se empleará el proyecto Singlethread-SIMD. Las extensiones SIMD se usarán para la implementación de las operaciones repetitivas que se llevan a cabo durante el procesamiento de las imágenes indicado por tu profesor. A estas extensiones también se las conoce como extensiones multimedia o vectoriales, pues en este campo de aplicación es habitual la realización de operaciones repetitivas muy simples sobre grandes vectores o matrices que codifican imágenes, audio y vídeo.

Las extensiones multimedia son opcionales y se agrupan en conjuntos de instrucciones. Dependiendo del procesador que incorpore el equipo pueden estar disponibles algunas y otras no. Por esta razón, en el trabajo a realizar en primer lugar se debe proporcionar una lista de las extensiones SIMD soportadas.

El empleo de las extensiones SIMD se lleva a cabo empleando lo que se denomina funciones intrínsecas (intrinsics). En apariencia son funciones de C, pero en realidad no lo son pues no son llamadas como las funciones habituales. Cada referencia a una función intrínseca en el código se traduce directamente en una instrucción ensamblador.

> **Extensiones SIMD soportadas:** MMX, SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2, AVX.

### Estructuras de datos para imágenes

Las imágenes se cargan en el sistema con la libreria `CImg` y se emplea el tipo `float` para representarlas. Usando esta librería podemos obtener un único vector que contiene todos los datos de la imágen. El formato del vector para una imágen RGB sería:

$$v = [ r_{1}, r_{2}, r_{3},...,r_{n}, g_{1}, g_{2}, g_{3},...,g_{n}, b_{1}, b_{2}, b_{3},...,b_{n}]$$

Por lo tanto lo que tenemos es un único vector que contiene el valor de las componentes rojas, luego verdes y finalmente azules. De lo que se deduce que el tamaño de $v$ es: $ancho \cdot alto \cdot númeroComponentes$.

### Algoritmo a implementar

El algoritmo asignado es el de fusionar dos imágenes con el modo amplitud. Para conseguir el modo amplitud los vectores R' G' y B', que representan las componentes de la imagen resultante, se calculan de la siguente manera:

$${R'} = \frac{\sqrt{Ra^{2} + Rb^{2}}}{\sqrt{2}}$$

$${G'} = \frac{\sqrt{Ga^{2} + Gb^{2}}}{\sqrt{2}}$$

$${B'} = \frac{\sqrt{Ba^{2} + Bb^{2}}}{\sqrt{2}}$$

De las fórmulas anteriores se puede deducir que para cualquier coordenada RGB de las imágenes 1 y 2 la fórmula de transformación será:

$$(x{_{3}}, y{_{3}}, z{_{3}}) = \left ( \frac{\sqrt{x{_{1}}^{2} + x{_{2}}^{2}}}{\sqrt{2}},  \frac{\sqrt{y{_{1}}^{2} + y{_{2}}^{2}}}{\sqrt{2}}, \frac{\sqrt{z{_{1}}^{2} + z{_{2}}^{2}}}{\sqrt{2}} \right )$$

Siendo $x{_{3}}, y{_{3}}, z{_{3}}$ las coordenadas RGB de la imágen resultante de la fusión con el modo amplitud de las imágenes 1 y 2.

### Algoritmo implementado

Cómo hemos visto las imágenes se encuentran representadas por medio de vectores que contienen sus componentes, por lo tanto, tendremos:

$imagen_{1} = [ r_{11}, r_{12}, r_{13},...,r_{1n}, g_{11}, g_{12}, g_{13},...,g_{1n}, b_{11}, b_{12}, b_{13},...,b_{1n}]$

$imagen_{2} = [ r_{21}, r_{22}, r_{23},...,r_{2n}, g_{21}, g_{22}, g_{23},...,g_{2n}, b_{21}, b_{22}, b_{23},...,b_{2n}]$

$imagen_{3} = [imagen_{1}.ancho \cdot imagen_{1}.alto \cdot imagen_{1}.númeroComponentes]$


PARA CADA i DESDE 0 HASTA $(imagen_{1}.ancho \cdot imagen_{1}.alto \cdot imagen_{1}.númeroComponentes)$
    
$$imagen_{3i} = \frac{\sqrt{imagen_{1i}^{2} + imagen_{2i}^{2}}}{\sqrt{2}}$$

Teniendo en cuanta las intrucciones SIMD que soporta el procesador en el que se implementa el algoritmo se usará la version de AVX, ya que es la más moderna de las intrucciones SIMD soportada.

Con las instrucciones SIMD podemos realizar operaciones sobre vectores de `SIMD_BANDWITH` elementos en una sola operación con lo que recorreremos el vector de `SIMD_BANDWIT` en `SIMD_BANDWIT` elementos.

Por lo tanto y para poder aplicar la fórmula anterior a `SIMD_BANDWITH` elementos al unisono debemos de descomponer la fórmula en operaciones elementales. El siguiente código ilustra una versión descompuesta del algoritmo implementado:

```c++
for (int i = 0; i < size; i += SIMD_BANDWITH) {   
    a = _mm256_loadu_ps(&pcompImage1[i]); // cargar img1
    b = _mm256_loadu_ps(&pcompImage2[i]); // cargar img2
    a2 = _mm256_mul_ps(a, a); // img1^2
    b2 = _mm256_mul_ps(b, b); // img2^2
    ab2 = _mm256_add_ps(a2, b2); // img1^2 + img2^2
    raizab2 = _mm256_sqrt_ps(ab2); // raiz_cuadrada( img1^2 + img2^2 )
    res8 = _mm256_div_ps(raizab2, vsqrt2); // raiz_cuadrada( img1^2 + img2^2 ) / raiz_cuadrada( 2.0 )
    
    _mm256_storeu_ps(&pdstImage[i], res8); // img3 = raiz_cuadrada( img1^2 + img2^2 ) / raiz_cuadrada( 2.0 )
}

```

El siguiente código muestra el algoritmo que se implementó:

```c++
for (int i = 0; i < size; i += SIMD_BANDWITH) {   
    _mm256_storeu_ps(
        &pdstImage[i],
        _mm256_div_ps(
            _mm256_sqrt_ps(
                _mm256_add_ps(
                    _mm256_mul_ps(
                        _mm256_loadu_ps(&pcompImage1[i]),
                        _mm256_loadu_ps(&pcompImage1[i])),
                    _mm256_mul_ps(
                        _mm256_loadu_ps(&pcompImage2[i]),
                        _mm256_loadu_ps(&pcompImage2[i])
                    )
                )
            ),
            vsqrt2
        )
    );
}

```

> **Nota**: _Como la ejecucion del algoritmo dura menos de 5 segundos se anida el anterior algoritmo dentro de un bucle for que lo repetirá 40 veces con lo que el tiempo de la ejecución del programa será superior a los 5 segundos, **pero se estará ejecutando el algoritmo 40 veces**._

> **Nota:** El algoritmo anterior se encuentra implementado únicamente con instrucciones SIMD anidadas, sin variables.

### Análisis del algoritmo
Para la ejecución del algoritmo anterior se obtienen los siguientes datos tras realizar 10 ejecuciones en modo release:

In [1]:
import pandas as pd
data = pd.Series([5.2364, 5.2364, 5.2364, 5.2364, 5.2364, 5.2364, 5.2364, 5.2364, 5.2364, 5.2364],
                 index=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
table = pd.DataFrame({'Duración':data})
table

,Duración
1,5.2364
2,5.2364
3,5.2364
4,5.2364
5,5.2364
6,5.2364
7,5.2364
8,5.2364
9,5.2364
10,5.2364


In [2]:
import numpy as np
mean = np.mean(data)
std = np.std(data)
print("Media 40 ejecuciones: ", mean, "s.")
print("Desviación estándar: ", std, ".")
print("Media 1 ejecución: ", mean/40.0, "s."); # 40 es el número de ejecuciones.

Media 40 ejecuciones:  5.2364 s.
Desviación estándar:  8.881784197e-16 .
Media 1 ejecución:  0.13091 s.


#### Comparación con versión monohilo sin instrucciones SIMD

A continuación se muestra la acceleración obtenida de esta versión con respecto a la versión monohilo y sin instrucciones SIMD. Las mejoras de esta versió que pueden afectar a la aceleración son:

    - Único bucle para recorrer vector entero.
    - Reducción en el uso de punteros.
    - Implementación de instrucciones SIMD con ancho de banda de 256 bit.
    

In [3]:
import pandas as pd
data = pd.Series([3.1182, 0.13091],
                 index=['monohilo', 'simd']);
table = pd.DataFrame({'T. x 1 Ejecución':data})
table

,T. x 1 Ejecución
monohilo,3.11820
simd,0.13091


In [4]:
mean_monohilo = 3.1182;
mean_simd = 0.13091
print("Aceleración: ", mean_monohilo/mean_simd , "s.");

Aceleración:  23.819417920708883 s.
